bug : learning rate = 10.0 instead of 1 or 0.1 or 0.01 or 0.001

In [1]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    delta_score_matrix = np.zeros((num_clients, num_clients))
    p_ks_matrix = np.zeros((num_clients, num_clients))
    p_x2_matrix = np.zeros((num_clients, num_clients))
    
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, Δ_score, P_KS, P_X2 = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class
            delta_score_matrix[i, j] = Δ_score
            delta_score_matrix[j, i] = Δ_score
            p_ks_matrix[i, j] = P_KS
            p_ks_matrix[j, i] = P_KS
            p_x2_matrix[i, j] = P_X2
            p_x2_matrix[j, i] = P_X2
    
    # Print the matrices
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    print(f"Round {round_num} Delta Score Matrix:")
    print(delta_score_matrix)
    print(f"Round {round_num} P_KS Matrix:")
    print(p_ks_matrix)
    print(f"Round {round_num} P_X2 Matrix:")
    print(p_x2_matrix)
    
    # Combine multiple matrices for the final distance calculation
    combined_matrix = delta_class_matrix + delta_score_matrix + p_ks_matrix + p_x2_matrix
    distance_matrix = np.linalg.norm(combined_matrix[:, np.newaxis] - combined_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Combined Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.84071666), ('loss', 0.5791551)]))])
Round 1 Delta Class Matrix:
[[   0. 8863. 8889. 8849. 8846. 8919. 8962. 8877. 8921. 8900.]
 [8863.    0. 1928. 2040. 2110. 2075. 2120. 2010. 2101. 2018.]
 [8889. 1928.    0. 1868. 1888. 1779. 1877. 1828. 1914. 1803.]
 [8849. 2040. 1868.    0. 2112. 2002. 1907. 1855. 2022. 1912.]
 [8846. 2110. 1888. 2112.    0. 1851. 1943. 1829. 1985. 1898.]
 [8919. 2075. 1779. 2002. 1851.    0. 1724. 1738. 1777. 1814.]
 [8962. 2120. 1877. 1907. 1943. 1724.    0. 1794. 1967. 1863.]
 [8877. 2010. 1828. 1855. 1829. 1738. 1794.    0. 1938. 1845.]
 [8921. 2101. 1914. 2022. 1985. 1777. 1967. 1938.    0. 1826.]
 [8900. 2018. 1803. 1912. 1898. 1814. 1863. 1845. 1826.    0.]]
Round 1 Delta Score Matrix:
[[     0. 100000. 100000. 100000. 100000. 100000. 100000. 100000. 100000.
  100000.

bug : Client 0 is the only one with the normalization bug (preprocess_fn_with_bug).

Bug Injection: Ensure the bug only applies to client 0 by correctly invoking preprocess_fn_with_bug for this client.